# Merge

Branches merge is opertion that allows you to get changes from intependend branches in one common commit. The `git merge` command is crucial for this page.

To merge branches you need to `git checkout` to brunch to which we merge all the changes, and execute command `git merge <branch to be merged>`.

## To ancestor

The simplest case is when you create a branch, make a few commits, and want to add those changes from an ancestor branch that hasn't changed.

Simply put, you just tell the parent branch to just start referencing the last commit of the branch you want to merge into the parent branch.

In the next cell:

- Created repository where there is `basic commit`;
- From `basic commit` branch `example_branch` was created;
- `branch_commit` was commited to `example_branch`, first `git log` display exactly this state of the repository;
- Then `example_branch` was merged to `master` - on `git log` it just desplayed as `master` moved to same commit as `example_branch`.

In [18]:
%%bash
mkdir merge_example
cd merge_example
git init &> /dev/null

echo "content" > test_file
git add --all
git commit -m "basic commit" &> /dev/null

echo
echo "=====creating branch====="
git checkout -b example_branch
echo "content2" > test_file
git commit -am "branch commit" &> /dev/null
git log --decorate --graph

echo
echo "=====merging to master====="
git checkout master

echo "-----git merge-----"
git merge example_branch
echo "-----git log-----"
git log --decorate --graph

cd ..
rm -r merge_example


=====creating branch=====


Switched to a new branch 'example_branch'


* commit 73fb590f3c60ce8841035c4381714dcb90cbb035 (HEAD -> example_branch)
| Author: Fedor Kobak <kobfedsur@gmail.com>
| Date:   Sun Sep 10 12:34:53 2023 +0300
| 
|     branch commit
| 
* commit 1af04e597f14270202f4625baee8c10d82a24651 (master)
  Author: Fedor Kobak <kobfedsur@gmail.com>
  Date:   Sun Sep 10 12:34:53 2023 +0300
  
      basic commit

=====merging to master=====
-----git merge-----
Updating 1af04e5..73fb590
Fast-forward


Switched to branch 'master'


 test_file | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
-----git log-----
* commit 73fb590f3c60ce8841035c4381714dcb90cbb035 (HEAD -> master, example_branch)
| Author: Fedor Kobak <kobfedsur@gmail.com>
| Date:   Sun Sep 10 12:34:53 2023 +0300
| 
|     branch commit
| 
* commit 1af04e597f14270202f4625baee8c10d82a24651
  Author: Fedor Kobak <kobfedsur@gmail.com>
  Date:   Sun Sep 10 12:34:53 2023 +0300
  
      basic commit
